In [2]:
import sys
sys.path.append("..")

import SimpleITK as sitk
from utils.plot import plot_volume, plot
from utils.dicom import resample, normalize_hu, read_dcm
from utils.ct import project, to_8bit, min_max_normalize
from covid_ct.dataset.body_mask import get_body_mask
from covid_ct.metadata.parse_dcm_fields import parse_record
from covid_ct.metadata.select_dcms import check_series_consistency, check_unique, classify_images
import numpy as np
from tqdm import tqdm
import pandas as pd
import pydicom as pyd
from typing import Dict, Any
from utils.config import CONFIG


In [3]:
df = pd.read_csv(CONFIG.OUTPUT_DIR / "lidc_meta.csv")

In [1]:
items = pd.read_pickle("/workspace/output/lidc/train/metadata.pkl")
items.head()

NameError: name 'pd' is not defined

In [8]:
df = pd.read_csv(CONFIG.OUTPUT_DIR / "lidc_meta.csv")
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)
pd.set_option('display.width', 99999)
# pd.set_option('display.max_colwidth', 99999)
df = df.query("PatientPosition == 'FFS' & ImageType0 == 'ORIGINAL' & ImageType1 == 'PRIMARY'")
df.groupby(["ImageOrientationPatient"])[["Subject", "Path"]].nunique()

,Subject,Path
ImageOrientationPatient,,
"[1, 0, 0, 0, 1, 0]",230,54968
"[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",653,151048


In [5]:
df[df["Subject"] == 966].sort_values(by="Path")

,Subject,ID,Path,PhotometricInterpretation,SliceLocation,SliceThickness,ImagePositionPatient0,ImagePositionPatient1,ImagePositionPatient2,PatientPosition,ImageOrientationPatient,SeriesDescription,SeriesNumber,PixelSpacing0,PixelSpacing1,ImageType0,ImageType1,ImageType2,ImageType3,HighBit
120344,966,1-001,/data/lidc/manifest-1600709154662/LIDC-IDRI/LI...,MONOCHROME2,7.420000,2.5,-179.699997,-175.0,7.420000,FFS,"[1.000000, 0.000000, 0.000000, 0.000000, 1.000...",NaN,3000280.0,0.683594,0.683594,ORIGINAL,PRIMARY,AXIAL,NONE,15
120401,966,1-002,/data/lidc/manifest-1600709154662/LIDC-IDRI/LI...,MONOCHROME2,5.420000,2.5,-179.699997,-175.0,5.420000,FFS,"[1.000000, 0.000000, 0.000000, 0.000000, 1.000...",NaN,3000280.0,0.683594,0.683594,ORIGINAL,PRIMARY,AXIAL,NONE,15
120374,966,1-003,/data/lidc/manifest-1600709154662/LIDC-IDRI/LI...,MONOCHROME2,3.420000,2.5,-179.699997,-175.0,3.420000,FFS,"[1.000000, 0.000000, 0.000000, 0.000000, 1.000...",NaN,3000280.0,0.683594,0.683594,ORIGINAL,PRIMARY,AXIAL,NONE,15
120323,966,1-004,/data/lidc/manifest-1600709154662/LIDC-IDRI/LI...,MONOCHROME2,1.420000,2.5,-179.699997,-175.0,1.420000,FFS,"[1.000000, 0.000000, 0.000000, 0.000000, 1.000...",NaN,3000280.0,0.683594,0.683594,ORIGINAL,PRIMARY,AXIAL,NONE,15
120386,966,1-005,/data/lidc/manifest-1600709154662/LIDC-IDRI/LI...,MONOCHROME2,-0.580000,2.5,-179.699997,-175.0,-0.580000,FFS,"[1.000000, 0.000000, 0.000000, 0.000000, 1.000...",NaN,3000280.0,0.683594,0.683594,ORIGINAL,PRIMARY,AXIAL,NONE,15
120294,966,1-006,/data/lidc/manifest-1600709154662/LIDC-IDRI/LI...,MONOCHROME2,-2.580000,2.5,-179.699997,-175.0,-2.580000,FFS,"[1.000000, 0.000000, 0.000000, 0.000000, 1.000...",NaN,3000280.0,0.683594,0.683594,ORIGINAL,PRIMARY,AXIAL,NONE,15
120370,966,1-007,/data/lidc/manifest-1600709154662/LIDC-IDRI/LI...,MONOCHROME2,-4.580000,2.5,-179.699997,-175.0,-4.580000,FFS,"[1.000000, 0.000000, 0.000000, 0.000000, 1.000...",NaN,3000280.0,0.683594,0.683594,ORIGINAL,PRIMARY,AXIAL,NONE,15
120396,966,1-008,/data/lidc/manifest-1600709154662/LIDC-IDRI/LI...,MONOCHROME2,-6.580000,2.5,-179.699997,-175.0,-6.580000,FFS,"[1.000000, 0.000000, 0.000000, 0.000000, 1.000...",NaN,3000280.0,0.683594,0.683594,ORIGINAL,PRIMARY,AXIAL,NONE,15
120424,966,1-009,/data/lidc/manifest-1600709154662/LIDC-IDRI/LI...,MONOCHROME2,-8.580000,2.5,-179.699997,-175.0,-8.580000,FFS,"[1.000000, 0.000000, 0.000000, 0.000000, 1.000...",NaN,3000280.0,0.683594,0.683594,ORIGINAL,PRIMARY,AXIAL,NONE,15
120420,966,1-010,/data/lidc/manifest-1600709154662/LIDC-IDRI/LI...,MONOCHROME2,-10.580000,2.5,-179.699997,-175.0,-10.580000,FFS,"[1.000000, 0.000000, 0.000000, 0.000000, 1.000...",NaN,3000280.0,0.683594,0.683594,ORIGINAL,PRIMARY,AXIAL,NONE,15


In [10]:
df_count = df.groupby(["Subject", "SeriesNumber"]).count()
df_count[df_count["Path"] < 100]

,,ID,Path,PhotometricInterpretation,SliceLocation,SliceThickness,ImagePositionPatient0,ImagePositionPatient1,ImagePositionPatient2,PatientPosition,ImageOrientationPatient,SeriesDescription,PixelSpacing0,PixelSpacing1,ImageType0,ImageType1,ImageType2,ImageType3,HighBit
Subject,SeriesNumber,,,,,,,,,,,,,,,,,,
365,5409.0,98,98,98,98,98,98,98,98,98,98,0,98,98,98,98,98,98,98


In [59]:
df[df["Subject"] == 678]

,Subject,ID,Path,PhotometricInterpretation,SliceLocation,SliceThickness,ImagePositionPatient0,ImagePositionPatient1,ImagePositionPatient2,PatientPosition,ImageOrientationPatient,SeriesDescription,SeriesNumber,PixelSpacing0,PixelSpacing1,ImageType0,ImageType1,ImageType2,ImageType3,HighBit
70346,678,1-010,/data/lidc/manifest-1600709154662/LIDC-IDRI/LIDC-IDRI-0678/01-01-2000-NA-NA-72850/3000016.000000-NA-10450/1-010.dcm,MONOCHROME2,9.6,2.5,-149.0,-135.0,9.6,FFS,"[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",NaN,3000016.0,0.527344,0.527344,ORIGINAL,PRIMARY,AXIAL,NONE,15
